In [8]:
import pandas as pd
import random
from pathlib import Path
import numpy as np

In [9]:
directory = Path("archive/")
books = pd.concat([pd.read_csv(f).drop(
    ['RatingDist1', 'RatingDist4', 'RatingDistTotal', 'RatingDist2', 'RatingDist5', 'RatingDist3', 'Id'], axis=1) for f in directory.glob("*.csv")], ignore_index=True)

# books = books.drop(['pagesNumber', 'Count of text reviews'], axis=1)
books.Language = books.Language.fillna('eng')
books.pagesNumber = books.pagesNumber.fillna(random.randint(250, 1500))
books.pagesNumber = books.pagesNumber.astype('int')
books.Publisher = books.Publisher.fillna('Alpina')
books['Genre'] = np.random.choice(
    ['Action and Adventure', 'Classics', 'Comic Book', 'Detective', 'Mystery', 'Fantasy', 'Historical Fiction',
     'Literary Fiction', 'Horror', 'Romance', 'Science Fiction', 'Short Stories', 'Suspense', 'Thrillers',
     'Biographies', 'Autobiographies', 'Cookbooks', 'Essays', 'History', 'Memoir', 'Poetry', 'Self-Help', 'True Crime'],
    len(books))

books.loc[books['PublishYear'] > 2023, 'PublishYear'] = random.randint(1900, 2023)
books.loc[books['PublishMonth'] > 12, 'PublishMonth'] = random.randint(1, 12)
books.loc[books['PublishDay'] > 31, 'PublishDay'] = random.randint(1, 31)
books.loc[(books['Language'] == "en-US") | (books['Language'] == "en-GB") | (books['Language'] == "en-CA") | (books['Language'] == "--"), 'Language'] = "eng"

In [10]:
print(books.columns.values)

['Name' 'pagesNumber' 'PublishMonth' 'PublishDay' 'Publisher'
 'CountsOfReview' 'PublishYear' 'Language' 'Authors' 'Rating' 'ISBN'
 'Genre']


In [11]:
books.isnull().sum()

Name                0
pagesNumber         0
PublishMonth        0
PublishDay          0
Publisher           0
CountsOfReview      0
PublishYear         0
Language            0
Authors             0
Rating              0
ISBN              546
Genre               0
dtype: int64

In [12]:
books.nunique()

Name              57510
pagesNumber        1340
PublishMonth         12
PublishDay           12
Publisher          7521
CountsOfReview     2163
PublishYear         105
Language             33
Authors           28633
Rating              267
ISBN              57552
Genre                23
dtype: int64

In [13]:
print(books.Authors.unique())
print("-"*40)
print(books.Publisher.unique())
print("-"*40)
print(books.Language.unique())
print("-"*40)
print(books.Genre.unique())

['J.K. Rowling' 'W. Frederick Zimmerman' 'Douglas Adams' ...
 'Jonathan Weiner' 'Archibald Hamilton Rutledge' "Daniel O'Connor"]
----------------------------------------
['Scholastic Inc.' 'Scholastic Inc' 'Scholastic' ... 'Twelve'
 'Fantastic Audio' 'Gray & Company Publishers']
----------------------------------------
['eng' 'fre' 'spa' 'mul' 'grc' 'enm' 'ger' 'jpn' 'ara' 'nl' 'zho' 'lat'
 'por' 'srp' 'ita' 'rus' 'msa' 'glg' 'wel' 'swe' 'nor' 'kor' 'tur' 'gla'
 'lit' 'per' 'pol' 'gle' 'cat' 'afr' 'ind' 'frs' 'sco']
----------------------------------------
['Mystery' 'Suspense' 'Poetry' 'Romance' 'Classics' 'Historical Fiction'
 'Autobiographies' 'Horror' 'Thrillers' 'Self-Help' 'Biographies'
 'True Crime' 'Memoir' 'History' 'Science Fiction' 'Short Stories'
 'Detective' 'Action and Adventure' 'Comic Book' 'Essays' 'Fantasy'
 'Cookbooks' 'Literary Fiction']


In [7]:
Book_genre = pd.DataFrame(data=books.Genre.unique(), columns=['Genre'])
Book_lang = pd.DataFrame(data=books.Language.unique(), columns=['Language'])
Book_publisher = pd.DataFrame(data=books.Publisher.unique(), columns=['Publisher'])
Book_author = pd.DataFrame(data=books.Authors.unique(), columns=['Authors'])

columns = [[Book_genre, "Genre"], [Book_lang, "Language"], [Book_publisher, "Publisher"], [Book_author, "Authors"]]
for col in columns:
    for index, string in enumerate(col[0].iloc[:, 0]):
        books.loc[books[col[1]] == string, col[1]] = index

KeyboardInterrupt: 

In [14]:
books

,Name,pagesNumber,PublishMonth,PublishDay,Publisher,CountsOfReview,PublishYear,Language,Authors,Rating,ISBN,Genre
0,Harry Potter and the Half-Blood Prince (Harry ...,652,10,9,Scholastic Inc.,28062,2006,eng,J.K. Rowling,4.57,NaN,Mystery
1,Harry Potter and the Order of the Phoenix (Har...,870,1,9,Scholastic Inc.,29770,2004,eng,J.K. Rowling,4.50,0439358078,Suspense
2,Harry Potter and the Sorcerer's Stone (Harry P...,309,1,11,Scholastic Inc,75911,2003,eng,J.K. Rowling,4.47,NaN,Poetry
3,Harry Potter and the Chamber of Secrets (Harry...,352,1,11,Scholastic,244,2003,eng,J.K. Rowling,4.42,0439554896,Romance
4,Harry Potter and the Prisoner of Azkaban (Harr...,435,1,5,Scholastic Inc.,37093,2004,eng,J.K. Rowling,4.57,043965548X,Suspense
...,...,...,...,...,...,...,...,...,...,...,...,...
58287,The Best American Science and Nature Writing 2000,265,10,10,Mariner Books,7,2000,eng,David Quammen,3.99,0618082956,Autobiographies
58288,The Best American Science and Nature Writing 2002,336,10,10,Mariner Books,7,2002,eng,Natalie Angier,4.11,0618134786,Comic Book
58289,Hunting and Home in the Southern Heartland: Th...,263,1,4,University of South Carolina Press,0,1992,eng,Archibald Hamilton Rutledge,4.30,0872498220,Thrillers
58290,The Best American Science Writing 2004,288,10,9,Ecco,5,2004,eng,Dava Sobel,3.86,0060726407,Suspense


In [ ]:
books.to_csv("archive/result.csv", index=False)